<a href="https://colab.research.google.com/github/CoderJeel/chest-xray-phenumonia-classfication/blob/master/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Flatten,Conv2D,MaxPooling2D,Dense,Dropout,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint
from tensorflow.keras.regularizers import Regularizer,L1,l2

In [2]:
import pickle
feature = pickle.load(open('/content/drive/My Drive/Projects/phenumonia detector/Training/feature_train.pickle','rb'))
labels = pickle.load(open('/content/drive/My Drive/Projects/phenumonia detector/Training/labels_train.pickle','rb'))


In [3]:
Val_feature = pickle.load(open('/content/drive/My Drive/Projects/phenumonia detector/Validation/feature_val.pickle','rb'))
Val_lab = pickle.load(open('/content/drive/My Drive/Projects/phenumonia detector/Validation/labels_val.pickle','rb'))

In [4]:
type(Val_feature) 

numpy.ndarray

In [ ]:
#feature[5]

In [5]:
print(feature.shape)
print(labels.shape)

(5216, 160, 200, 1)
(5216,)


In [8]:
#import time
#NAME = "PHENUMONIA-CLASSFICATION-{}".format(int(time.time()))
#tensorboard = TensorBoard(log_dir='Logs/{}'.format(NAME)) 

In [ ]:
dense_layer=[2,3,4,5]
layer_size = [32,64,128]
conv_layers = [1,2,3]
for dense in dense_layer:
  for layer in layer_size:
    for conv in conv_layers:
      Name = "{}-conv-{}-node-{}-dense-{}".format(dense,layer,conv,int(time.time()))
      print(Name)

In [9]:
!ls

drive  Logs  sample_data


In [ ]:
dense_layer=[2,3,4,5]
layer_size = [32,64,128]
conv_layers = [1,2,3]
for dense in dense_layer:
  for layer in layer_size:
    for conv in conv_layers:
      Name = "{}-conv-{}-node-{}-dense-{}".format(dense,layer,conv,int(time.time()))
      print(Name)
      tensorboard = TensorBoard(log_dir='Logs/{}'.format(Name)) 
      model = Sequential()
      model.add(Conv2D(layer,(3,3),input_shape=feature.shape[1:]))
      model.add(Activation('relu'))
      model.add(MaxPooling2D(pool_size=(2,2)))

      for l in range(conv-1):
        model.add(Conv2D(layer,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
      
      model.add(Flatten())
      
      for l in range(dense):
        model.add(Dense(layer))
        model.add(Activation('relu'))

      model.add(Dense(1))
      model.add(Activation('sigmoid'))

      model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

      model.fit(feature,labels,batch_size = 20,epochs=5,validation_split=0.1,callbacks=[tensorboard])

In [9]:
#model.save('/content/model_1_cvl.h5')

In [13]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [14]:
dense_layer=[2]
layer_size = [128]
conv_layers = [1]
for dense in dense_layer:
  for layer in layer_size:
    for conv in conv_layers:
      #Name = "{}-conv-{}-node-{}-dense-{}".format(dense,layer,conv,int(time.time()))
      #print(Name)
      #tensorboard = TensorBoard(log_dir='Logs/{}'.format(Name)) 
      model = Sequential()
      model.add(Conv2D(layer,(3,3),input_shape=feature.shape[1:]))
      model.add(Activation('relu'))
      model.add(MaxPooling2D(pool_size=(2,2)))

      for l in range(conv-1):
        model.add(Conv2D(layer,(3,3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
      
      model.add(Flatten())
      
      for l in range(dense):
        model.add(Dense(layer,kernel_regularizer=l2(0.001)))
        model.add(Activation('relu'))
        model.add(Dropout(0.2))

      model.add(Dense(1))
      model.add(Activation('sigmoid'))

      model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

      model.fit(feature,labels,batch_size =32,epochs=25,validation_data = (Val_feature,Val_lab),callbacks=[earlyStopping,mcp_save])

Epoch 1/25
163/163 [==============================] - 23s 142ms/step - loss: 1.1490 - accuracy: 0.8344 - val_loss: 1.1317 - val_accuracy: 0.6875
Epoch 2/25
163/163 [==============================] - 42s 257ms/step - loss: 0.3448 - accuracy: 0.9293 - val_loss: 0.4246 - val_accuracy: 0.8750
Epoch 3/25
163/163 [==============================] - 16s 100ms/step - loss: 0.2930 - accuracy: 0.9398 - val_loss: 0.6165 - val_accuracy: 0.7500
Epoch 4/25
163/163 [==============================] - 25s 151ms/step - loss: 0.2637 - accuracy: 0.9490 - val_loss: 0.3849 - val_accuracy: 0.8750
Epoch 5/25
163/163 [==============================] - 16s 99ms/step - loss: 0.2650 - accuracy: 0.9500 - val_loss: 0.4321 - val_accuracy: 0.8750
Epoch 6/25
163/163 [==============================] - 25s 151ms/step - loss: 0.2593 - accuracy: 0.9457 - val_loss: 0.2895 - val_accuracy: 1.0000
Epoch 7/25
163/163 [==============================] - 41s 254ms/step - loss: 0.2372 - accuracy: 0.9572 - val_loss: 0.1930 - val_acc

In [15]:
model.save('/content/model.h5')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Logs